In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [19]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


datasetPathGeneratedFaces =  configParser.get('evaluate_imagen', 'test_datasetPathGeneratedFaces')
generated_face_table_name =  configParser.get('evaluate_imagen', 'generated_face_table_name')
evaluation_result_file = configParser.get('evaluate_imagen', 'evaluation_result_file')


datasetPathDatabase =  configParser.get('COMMON', 'test_datasetPathDatabase') + '/dataset.db'



cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [3]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [4]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [5]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('evaluate_imagen', 'audio_length_used') 
model_filename =  configParser.get('evaluate_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('evaluate_imagen', 'openl3_mode')

In [6]:
import lancedb
uri = datasetPathDatabase
db = lancedb.connect(uri)

table = db.open_table("video_stage1")

In [7]:
try:
  db.drop_table(generated_face_table_name)
except:
  print("")

In [8]:
import multiprocessing
try:
    multiprocessing.set_start_method('spawn')
except:
    print("")

In [9]:
import pathlib
import os
from random import randrange


uri = datasetPathDatabase
db = lancedb.connect(uri)


files_to_insert = []


for root, dirs, files in os.walk(datasetPathGeneratedFaces):
    for file in files:
        files_to_insert.append(root +'/'+ file)
        #print(root + file)



In [10]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [11]:

from testing_imagen_vision_transformer import extract_vision_transformer
from testing_imagen_face import extract_face_rep,extract_face_attributes
import pathlib
import shutil


2023-10-06 21:40:24.758390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:


i = 0
for face_file in files_to_insert:
    
    q = multiprocessing.Queue()

    try:
        proc = multiprocessing.Process(target=extract_face_rep, args=(q,face_file,
                                                                image_size,output_folder,))
        proc.start()
        proc.join()
    except:
        continue

    with open(output_folder + '/' + 'vgg_generated_face.pickle', 'rb') as handle:
        vgg = pickle.load(handle)
        #print("++++++" + str(vgg))

    try:
        proc = multiprocessing.Process(target=extract_face_attributes, args=(q,face_file,))
        proc.start()
        proc.join()

        gender = q.get()
        if(gender == 'Error'):
            continue
        ethnicity = q.get()
        if(ethnicity == 'Error'):
            continue
        age = q.get()
        if(age == 'Error'):
            continue

    except:
        continue


    
    tbl = db.open_table("video_stage1")
    result = tbl.search(vgg).limit(10).to_df()

    match_1 = result.iloc[0]['user']
    match_2 = result.iloc[1]['user']
    match_3 = result.iloc[2]['user']
    match_4 = result.iloc[3]['user']
    match_5 = result.iloc[4]['user']
    match_6 = result.iloc[5]['user']
    match_7 = result.iloc[6]['user']
    match_8 = result.iloc[7]['user']
    match_9 = result.iloc[8]['user']
    match_10= result.iloc[9]['user']
    

    p_small = pathlib.Path(os.path.dirname(face_file))
    p_big = p_small.parent.absolute()
    
    df_table = pd.DataFrame()
    df_table = df_table.append({'id': i , 
                    'generated_face_path': face_file,'user':p_big.name,'age':age,'gender':gender,'ethnicity':ethnicity,
                    'match_1':match_1,'match_2':match_2,'match_3':match_3,'match_4':match_4,'match_5':match_5,
                    'match_6':match_6,'match_7':match_7,'match_8':match_8,'match_9':match_9,'match_10':match_10,
        'vector' : vgg,'stage': 1}, ignore_index=True)
    

    if(i == 0):
        try:
            db.create_table(generated_face_table_name, df_table)
        except:
            db.drop_table(generated_face_table_name)
            db.create_table(generated_face_table_name, df_table)
    else:
        tbl = db.open_table(generated_face_table_name)
        tbl.add(df_table)


    print("---------------" + str(i) + "----------------")
    i = i + 1

shutil.rmtree(output_folder)


---------------0----------------
---------------1----------------
---------------2----------------
---------------3----------------
---------------4----------------
---------------5----------------
---------------6----------------
---------------7----------------
---------------8----------------
---------------9----------------
---------------10----------------


In [13]:
tbl = db.open_table(generated_face_table_name)

In [15]:
ptable = tbl.to_pandas()
ptable

,id,generated_face_path,user,match_1,match_2,match_3,match_4,match_5,match_6,match_7,match_8,match_9,match_10,vector,stage
0,0,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00002,id00003,id00001,id00006,id00002,id00010,id00008,id00005,id00009,id00012,id00007,"[0.016634291, 0.00096485106, 0.014822065, 0.03...",1
1,1,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00001,id00003,id00001,id00009,id00007,id00010,id00004,id00008,id00002,id00006,id00012,"[0.014305992, 0.00968499, 0.015560359, 0.02682...",1
2,2,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00006,id00003,id00001,id00009,id00007,id00002,id00010,id00004,id00006,id00008,id00012,"[0.013197578, 0.004402566, 0.014329791, 0.0255...",1
3,3,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00008,id00003,id00001,id00009,id00010,id00007,id00008,id00006,id00002,id00004,id00012,"[0.012554283, 0.004316603, 0.011428701, 0.0281...",1
4,4,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00010,id00003,id00001,id00002,id00010,id00009,id00007,id00006,id00008,id00005,id00004,"[0.014646208, 0.0038225134, 0.01508191, 0.0265...",1
5,5,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00012,id00003,id00001,id00010,id00006,id00002,id00005,id00007,id00009,id00008,id00012,"[0.019030118, 0.007888322, 0.01813038, 0.03026...",1
6,6,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00009,id00003,id00006,id00010,id00012,id00001,id00009,id00008,id00007,id00002,id00005,"[0.013786207, 0.0091431085, 0.011618518, 0.035...",1
7,7,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00005,id00003,id00001,id00002,id00007,id00010,id00009,id00005,id00006,id00008,id00004,"[0.015202785, 0.008085675, 0.0192317, 0.027221...",1
8,8,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00004,id00003,id00001,id00010,id00006,id00009,id00002,id00007,id00008,id00004,id00005,"[0.016861234, 0.008097973, 0.012873291, 0.0244...",1
9,9,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00007,id00003,id00001,id00009,id00010,id00007,id00006,id00012,id00002,id00004,id00008,"[0.008715284, 0.008745547, 0.019739512, 0.0285...",1


In [24]:
ptable_unique_users = ptable.drop_duplicates(subset=['user'], keep='first')

In [60]:
f= open(evaluation_result_file,"w+")
f.write("No. of generated faces: " + str(len(ptable)) + "\n")
f.write("No. of unique users: " + str(len(ptable_unique_users)) + "\n")


24

In [61]:
acc_10 = int((len(tbl.search() \
   .where(""" user = match_1 or user = match_2 or user = match_3 
          or user = match_4 or user = match_5 or user = match_6 or user = match_7
          or user = match_8 or user = match_9 or user = match_10
    """).to_df()) / float(len(ptable))) * 100)
f.write("Face Recognition Accuracy within first 10 matches: " + str(acc_10) + "%\n")


55

In [62]:
acc_5 = int((len(tbl.search() \
   .where(""" user = match_1 or user = match_2 or user = match_3
          or user = match_4 or user = match_5 
    """).to_df()) / float(len(ptable))) * 100)
f.write("Face Recognition Accuracy within first  5 matches: " + str(acc_5) + "%\n")

55

In [63]:
acc_3 = int((len(tbl.search() \
   .where(""" user = match_1 or user = match_2 or user = match_3
    """).to_df()) / float(len(ptable))) * 100)
f.write("Face Recognition Accuracy within first  3 matches: " + str(acc_3) + "%\n")

55

In [64]:
acc_1 = int((len(tbl.search() \
   .where(""" user = match_1
    """).to_df()) / float(len(ptable))) * 100)
f.write("Face Recognition Accuracy within first  1 matches: " + str(acc_1) + "%\n")

54

In [65]:
f.close()